# Stock Chart Pattern Recognition Using Deep learning

In [2]:

from cassandra.cluster import Cluster
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow / Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.preprocessing import MinMaxScaler

# HuggingFace Transformers สำหรับ sentiment
from transformers import pipeline

# 1️⃣ ดึงข้อมูล OHLCV จาก Cassandra

In [ ]:
def get_candlestick_data(symbol: str, limit: int = 3000):
    try:
        cluster = Cluster(['127.0.0.1'], port=9042)
        session = cluster.connect('stock_data')
        query = """
            SELECT time, open_price, high_price, low_price, close_price, volume
            FROM candlestick_data
            WHERE symbol = %s
            ORDER BY time DESC
            LIMIT %s
        """
        rows = session.execute(query, (symbol, limit))
        df = pd.DataFrame(list(rows))
        if df.empty:
            print(f"⚠️ ไม่พบข้อมูลสำหรับ symbol '{symbol}'")
            return pd.DataFrame()
        df.rename(columns={
            'open_price':'open', 'high_price':'high',
            'low_price':'low', 'close_price':'close'
        }, inplace=True)
        df.sort_values('time', inplace=True)
        df.reset_index(drop=True, inplace=True)
        return df
    except Exception as e:
        raise ConnectionRefusedError(f"❌ Cassandra error: {e}")
    finally:
        try: cluster.shutdown()
        except: pass

# 2️⃣ LSTM Price Prediction

In [4]:
def lstm_price_prediction(df: pd.DataFrame, seq_length=30):
    close = df['close'].values.reshape(-1,1)
    scaler = MinMaxScaler()
    close_scaled = scaler.fit_transform(close)

    # สร้าง sequence
    X, y = [], []
    for i in range(len(close_scaled)-seq_length):
        X.append(close_scaled[i:i+seq_length])
        y.append(close_scaled[i+seq_length])
    X, y = np.array(X), np.array(y)

    split = int(len(X)*0.8)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    model = Sequential()
    model.add(LSTM(64, input_shape=(seq_length,1)))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=1)

    y_pred = model.predict(X_test)
    y_pred_rescaled = scaler.inverse_transform(y_pred)
    return y_pred_rescaled, y_test

# 3️⃣ CNN Pattern Recognition (Head & Shoulders)

In [5]:
def generate_hs_labels(df: pd.DataFrame):
    df = df.copy()
    df['local_max'] = df['close'][(df['close'].shift(1)<df['close']) & (df['close'].shift(-1)<df['close'])]
    local_max_idx = df['local_max'].dropna().index
    labels = np.zeros(len(df))
    for i in range(len(local_max_idx)-2):
        ls_idx, head_idx, rs_idx = local_max_idx[i], local_max_idx[i+1], local_max_idx[i+2]
        ls_price, head_price, rs_price = df.loc[ls_idx,'close'], df.loc[head_idx,'close'], df.loc[rs_idx,'close']
        if head_price>ls_price and head_price>rs_price and abs(ls_price-rs_price)/ls_price<0.05:
            labels[ls_idx:rs_idx+1]=1
    return labels

def cnn_hs_recognition(df: pd.DataFrame, seq_length=30):
    labels = generate_hs_labels(df)
    data = df['close'].values.reshape(-1,1)
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)

    # สร้าง sequence windows
    X, y = [], []
    for i in range(len(data_scaled)-seq_length):
        X.append(data_scaled[i:i+seq_length])
        y.append(labels[i+seq_length])
    X, y = np.array(X), np.array(y)

    split = int(len(X)*0.8)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # CNN 1D model
    model = Sequential()
    model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(seq_length,1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=1)

    y_pred = model.predict(X_test).flatten()
    return y_pred, y_test, split

# 4️⃣ Sentiment Analysis (Transformer)

In [6]:
def sentiment_analysis(text_list):
    classifier = pipeline("sentiment-analysis")
    results = classifier(text_list)
    return results

# 5️⃣ Visualization Example

In [7]:
def plot_cnn_predictions(df, y_pred, split, seq_length=30):
    plt.figure(figsize=(15,6))
    plt.plot(df['time'], df['close'], label='Close Price')
    hs_idx = np.where(np.concatenate([np.zeros(seq_length+split), y_pred])>0.5)[0]
    plt.scatter(df.loc[hs_idx,'time'], df.loc[hs_idx,'close'], color='red', label='Predicted H&S')
    plt.title("Head & Shoulders Detection")
    plt.xlabel("Time")
    plt.ylabel("Close")
    plt.legend()
    plt.grid(True)
    plt.show()


# 6️⃣ MAIN PIPELINE

In [8]:
if __name__ == "__main__":
    symbol = 'BANPU'
    df = get_candlestick_data(symbol, limit=500)
    if df.empty:
        print("No data from Cassandra")
    else:
        # 1️⃣ LSTM Price Prediction
        y_pred_price, y_test_price = lstm_price_prediction(df)
        print("✅ LSTM price prediction done")

        # 2️⃣ CNN H&S Recognition
        y_pred_hs, y_test_hs, split = cnn_hs_recognition(df)
        print("✅ CNN H&S pattern recognition done")
        plot_cnn_predictions(df, y_pred_hs, split)

        # 3️⃣ Sentiment Analysis (example)
        texts = ["Bitcoin hits new all-time high!", "Market crashes due to regulation fears."]
        results = sentiment_analysis(texts)
        print("✅ Sentiment Analysis:", results)


ConnectionRefusedError: ❌ Cassandra error: Error from server: code=2200 [Invalid query] message="Undefined column name time in table stock_data.candlestick_data"